<a href="https://colab.research.google.com/github/rhyeu/data-manipulation-in-python/blob/main/dfply_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 작성일자 : 2021.02.02
- 작성자 : 류성균

- reference : [dplyr-style Data Manipulation with Pipes in Python](https://towardsdatascience.com/dplyr-style-data-manipulation-with-pipes-in-python-380dcb137000)

In [ ]:
!pip install dfply

! pip install nycflights13

     |████████████████████████████████| 614kB 5.7MB/s 
     |████████████████████████████████| 8.7MB 5.5MB/s 
  Created wheel for nycflights13: filename=nycflights13-0.0.3-cp36-none-any.whl size=8732755 sha256=e080b238dec4f63047fbf2e7e9e480667bd9426780d1800a450a58950f3cd44b
  Stored in directory: /root/.cache/pip/wheels/50/fe/ad/c6adafdf6e59183110670b4b14fb582c20b82a0fed449348cb
Successfully built nycflights13


### Getting started
- 예제 데이터는 nycflights13 데이터 사용

In [ ]:
from dfply import *
import pandas as pd

from nycflights13 import flights
# flights is the combined, tidied data, but can also import individual pieces..
from nycflights13 import airports

### Data

In [ ]:
flight_data = flights

flight_data.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


### Piping 

### 'original_data' could be a pandas DataFrame.
- `pandas` 형테의 전처리
```
result_1 = transformation_1(original_data, *args, **kwargs)
result_2 = transformation_2(result_1, *args, **kwargs)
result_3 = transformation_3(result_2, *args, **kwargs)
.
.
.
final_result = transformation_n(result_n-1, *args, **kwargs) 
```

- `pipe` 연산자를 통해 코드가 깔끔해짐 (ex - `R`의 `dplyr` `%>%`연산자

```
final_result = original_data -->
                transformation_1(*args, **kwargs) -->
                transformation_2(*args, **kwargs) -->
                transformation_3(*args, **kwargs) -->
                .
                .
                .
                transformation_n(*args, **kwargs)
```
- `dfply` 패키지
    - 에서는 `>>` 연산자를 사용
    - 상속하는 컬럼에서는 `X.`를 붙여서 어느 데이터프레임의 컬럼인지를 표시하게끔 함

```
# 'data' is the original pandas DataFrame
(data >>
 select(X.first_col, X.second_col, X.third_col) >>
 drop(X.third_col) >>
 head(3))
 ```

### Exploring some of dfply’s transformation methods

- selecting and dropping columns
- subsetting and filtering rows
- grouping data
- reshaping data
- to name a few

#### Select and drop columns with `select()` and `drop()`

- 컬럼을 선택하려면 `select` 
- 컬럼을 제외하려면 `drop` 혹은 `select + ~`m


In [ ]:
(flight_data >> 
    select(X.origin, X.dest, X.hour))

,origin,dest,hour
0,EWR,IAH,5
1,LGA,IAH,5
2,JFK,MIA,5
3,JFK,BQN,5
4,LGA,ATL,6
...,...,...,...
336771,JFK,DCA,14
336772,LGA,SYR,22
336773,LGA,BNA,12
336774,LGA,CLE,11


In [ ]:
(flight_data >>
    drop(X.year, X.month, X.day))

,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,NaN,1455,NaN,NaN,1634,NaN,9E,3393,NaN,JFK,DCA,NaN,213,14,55,2013-09-30T18:00:00Z
336772,NaN,2200,NaN,NaN,2312,NaN,9E,3525,NaN,LGA,SYR,NaN,198,22,0,2013-10-01T02:00:00Z
336773,NaN,1210,NaN,NaN,1330,NaN,MQ,3461,N535MQ,LGA,BNA,NaN,764,12,10,2013-09-30T16:00:00Z
336774,NaN,1159,NaN,NaN,1344,NaN,MQ,3572,N511MQ,LGA,CLE,NaN,419,11,59,2013-09-30T15:00:00Z


In [ ]:
(flight_data >>
    select(~X.hour, ~X.minute))

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,3393,NaN,JFK,DCA,NaN,213,2013-09-30T18:00:00Z
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,3525,NaN,LGA,SYR,NaN,198,2013-10-01T02:00:00Z
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,3461,N535MQ,LGA,BNA,NaN,764,2013-09-30T16:00:00Z
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,3572,N511MQ,LGA,CLE,NaN,419,2013-09-30T15:00:00Z


#### Filter rows with mask()

In [ ]:
(flight_data >>
    mask(X.month == 1, X.day == 1, X.origin == 'JFK', X.hour > 10))

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
151,2013,1,1,848.0,1835,853.0,1001.0,1950,851.0,MQ,3944,N942MQ,JFK,BWI,41.0,184,18,35,2013-01-01T23:00:00Z
258,2013,1,1,1059.0,1100,-1.0,1210.0,1215,-5.0,MQ,3792,N509MQ,JFK,DCA,50.0,213,11,0,2013-01-01T16:00:00Z
265,2013,1,1,1111.0,1115,-4.0,1222.0,1226,-4.0,B6,24,N279JB,JFK,BTV,52.0,266,11,15,2013-01-01T16:00:00Z
266,2013,1,1,1112.0,1100,12.0,1440.0,1438,2.0,UA,285,N517UA,JFK,SFO,364.0,2586,11,0,2013-01-01T16:00:00Z
272,2013,1,1,1124.0,1100,24.0,1435.0,1431,4.0,B6,641,N590JB,JFK,SFO,349.0,2586,11,0,2013-01-01T16:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,2013,1,1,2326.0,2130,116.0,131.0,18,73.0,B6,199,N594JB,JFK,LAS,290.0,2248,21,30,2013-01-02T02:00:00Z
833,2013,1,1,2327.0,2250,37.0,32.0,2359,33.0,B6,22,N639JB,JFK,SYR,45.0,209,22,50,2013-01-02T03:00:00Z
835,2013,1,1,2353.0,2359,-6.0,425.0,445,-20.0,B6,739,N591JB,JFK,PSE,195.0,1617,23,59,2013-01-02T04:00:00Z
836,2013,1,1,2353.0,2359,-6.0,418.0,442,-24.0,B6,707,N794JB,JFK,SJU,185.0,1598,23,59,2013-01-02T04:00:00Z


#### Sort rows with `arrange()`
- 역순으로 할 때는 `ascending = False` argument 사용 

In [ ]:
(flight_data >> 
    arrange(X.distance, X.hour, ascemnding = [False, True]))

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
162,2013,1,1,857.0,900,-3.0,1516.0,1530,-14.0,HA,51,N380HA,JFK,HNL,659.0,4983,9,0,2013-01-01T14:00:00Z
1073,2013,1,2,909.0,900,9.0,1525.0,1530,-5.0,HA,51,N380HA,JFK,HNL,638.0,4983,9,0,2013-01-02T14:00:00Z
2018,2013,1,3,914.0,900,14.0,1504.0,1530,-26.0,HA,51,N380HA,JFK,HNL,616.0,4983,9,0,2013-01-03T14:00:00Z
2922,2013,1,4,900.0,900,0.0,1516.0,1530,-14.0,HA,51,N384HA,JFK,HNL,639.0,4983,9,0,2013-01-04T14:00:00Z
3791,2013,1,5,858.0,900,-2.0,1519.0,1530,-11.0,HA,51,N381HA,JFK,HNL,635.0,4983,9,0,2013-01-05T14:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117903,2013,2,8,NaN,2129,NaN,NaN,2224,NaN,EV,4619,N14952,EWR,PHL,NaN,80,21,29,2013-02-09T02:00:00Z
119764,2013,2,10,2127.0,2129,-2.0,2209.0,2224,-15.0,EV,4619,N41104,EWR,PHL,23.0,80,21,29,2013-02-11T02:00:00Z
120660,2013,2,11,2305.0,2129,96.0,3.0,2224,99.0,EV,4619,N11565,EWR,PHL,39.0,80,21,29,2013-02-12T02:00:00Z
121615,2013,2,12,2123.0,2130,-7.0,2211.0,2225,-14.0,EV,4619,N12921,EWR,PHL,21.0,80,21,30,2013-02-13T02:00:00Z


#### Add new columns with `mutate()`

In [ ]:
(flight_data >>
    mutate(
        new_distance = X.distance / 1000,
        carrier_origin = X.carrier + X.origin
    ) >>
    select(["new_distance", "carrier_origin"]))

,new_distance,carrier_origin
0,1.400,UAEWR
1,1.416,UALGA
2,1.089,AAJFK
3,1.576,B6JFK
4,0.762,DLLGA
...,...,...
336771,0.213,9EJFK
336772,0.198,9ELGA
336773,0.764,MQLGA
336774,0.419,MQLGA


### Group and ungroup data with `group_by()` and `ungroup()`


In [ ]:

(flight_data >>
    group_by(X.origin)
)


#### Sumarise data using `summarize()`

In [ ]:
(flight_data >>
 group_by(X.origin) >>
 summarize(mean_distance = X.distance.mean())
)

,origin,mean_distance
0,EWR,1056.742790
1,JFK,1266.249077
2,LGA,779.835671


#### 예제 : Bringing it all together with pipes
- [Step 1]: Filter out all flights less than 10 hours
- [Step 2]: Create a new column, speed, using the formula [distance / (air time * 60)]
- [Step 3]: Calculate the mean speed for flights originating from each airport
- [Step 4]: Sort the result by mean speed in descending order

In [ ]:
flight_data.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [ ]:
### pandas version
flight_data.loc[flight_data['hour']>10, 'speed'] = flight_data['distance'] / (flight_data['air_time'] * 60)
result = flight_data.groupby('origin', as_index = False)['speed'].mean()

result.sort_values('speed', ascending = False)

,origin,speed
0,EWR,0.109777
1,JFK,0.109427
2,LGA,0.107362


In [ ]:
### dfply version
(flight_data >>
    mask(X.hour > 10) >>
    mutate(speed = X.distance / (X.air_time * 60)) >>
    group_by(X.origin) >>
    summarise(mean_speed = mean(X.speed)) >>
    arrange(X.mean_speed, ascending = False)
)

,origin,mean_speed
0,EWR,0.109777
1,JFK,0.109427
2,LGA,0.107362
